Đây là kiến trúc CNN chúng ta sẽ xây dựng (dựa trên mô tả của bài báo ):

Input: Sequence dạng One-Hot Encoding (Ma trận 21 x Độ dài chuỗi).

Conv Layers: Nhiều bộ lọc với kích thước khác nhau (ví dụ: quét 8, 16, 24, 32 axit amin cùng lúc) để bắt các motif dài ngắn khác nhau.

Pooling: Max Pooling để chỉ giữ lại tín hiệu mạnh nhất của motif.

Output: Dự đoán nhãn.

## One-hot encoding

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import numpy as np
import pandas as pd
from Bio import SeqIO
from tqdm import tqdm
import gc

# --- CẤU HÌNH ---
TRAIN_FASTA = "/workspace/data/Train/train_sequences.fasta"
TRAIN_TERMS = "/workspace/data/Train/train_terms.tsv"
TEST_FASTA  = "/workspace/data/Test/testsuperset.fasta"
MODEL_PATH  = "deepgoplus_model.pth"

MAX_LEN = 1000   # DeepGOPlus thường dùng độ dài cố định (ví dụ 2000, ta dùng 1000 cho nhẹ)
BATCH_SIZE = 64  # Tăng batch size vì CNN nhẹ hơn Transformer nhiều
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY" # 20 loại axit amin chuẩn

# Map: Ký tự -> Số nguyên (1-20), 0 để padding
aa_to_id = {aa: i + 1 for i, aa in enumerate(AMINO_ACIDS)}

def get_one_hot_data(fasta_file, ids_filter=None):
    sequences = []
    ids = []
    print(f"⏳ Encoding {fasta_file}...")
    
    for record in tqdm(SeqIO.parse(fasta_file, "fasta")):
        pid = str(record.id).split("|")[1] if "|" in str(record.id) else str(record.id)
        
        if ids_filter is not None and pid not in ids_filter:
            continue
            
        seq = str(record.seq)
        # Encode sang số
        encoded_seq = [aa_to_id.get(aa, 0) for aa in seq[:MAX_LEN]]
        # Padding (điền số 0 vào đuôi cho đủ độ dài MAX_LEN)
        if len(encoded_seq) < MAX_LEN:
            encoded_seq += [0] * (MAX_LEN - len(encoded_seq))
            
        sequences.append(encoded_seq)
        ids.append(pid)
        
    return np.array(sequences, dtype=np.int32), ids

# --- LOAD DỮ LIỆU ---
# 1. Load Sequence
X_train_seq, train_ids = get_one_hot_data(TRAIN_FASTA)

# 2. Load Labels (Y) - Top 1500 nhãn phổ biến
print("⏳ Processing Labels...")
train_terms = pd.read_csv(TRAIN_TERMS, sep="\t", usecols=["EntryID", "term"])
TOP_N = 1500
top_terms = train_terms["term"].value_counts().head(TOP_N).index.tolist()
term_to_idx = {t: i for i, t in enumerate(top_terms)}

Y_train = np.zeros((len(train_ids), TOP_N), dtype=np.float32)
id_map = {pid: i for i, pid in enumerate(train_ids)}

# Fill Y matrix
grouped = train_terms[train_terms["term"].isin(top_terms)].groupby("EntryID")["term"].apply(list)
for pid, terms in tqdm(grouped.items()):
    if pid in id_map:
        indices = [term_to_idx[t] for t in terms]
        Y_train[id_map[pid], indices] = 1.0

print(f"✅ Data Ready: X={X_train_seq.shape}, Y={Y_train.shape}")

⏳ Encoding /workspace/data/Train/train_sequences.fasta...


82404it [00:01, 49812.91it/s]


⏳ Processing Labels...


76297it [00:00, 304256.58it/s]

✅ Data Ready: X=(82404, 1000), Y=(82404, 1500)


## Build model

In [2]:
class DeepGOPlus(nn.Module):
    def __init__(self, num_classes, vocab_size=21, embedding_dim=128, num_filters=512, kernel_sizes=[8, 16, 24, 32]):
        super(DeepGOPlus, self).__init__()
        
        # Lớp Embedding: Biến số nguyên (1, 2...) thành vector dày đặc
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # Các lớp Convolution song song (Multi-kernel)
        # Mỗi kernel size sẽ bắt các motif có độ dài khác nhau
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])
        
        # Lớp phân loại cuối cùng
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)
        self.dropout = nn.Dropout(0.5) # Chống học vẹt
        
    def forward(self, x):
        # x shape: [batch, seq_len]
        x = self.embedding(x)           # -> [batch, seq_len, emb_dim]
        x = x.permute(0, 2, 1)          # -> [batch, emb_dim, seq_len] (Pytorch Conv1d cần channel ở giữa)
        
        # Chạy qua từng bộ lọc Conv -> ReLU -> MaxPool
        outs = []
        for conv in self.convs:
            out = conv(x)               # Convolution
            out = torch.relu(out)       # Activation
            out, _ = torch.max(out, dim=2) # Global Max Pooling (Lấy tín hiệu mạnh nhất của motif)
            outs.append(out)
            
        # Nối các đặc trưng lại
        out = torch.cat(outs, dim=1)
        out = self.dropout(out)
        out = self.fc(out)
        return out # Trả về Logits (chưa qua Sigmoid)

# Khởi tạo
device = "cuda" if torch.cuda.is_available() else "cpu"
model = DeepGOPlus(num_classes=TOP_N).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

print("✅ Model CNN (DeepGOPlus style) đã sẵn sàng chiến đấu!")

✅ Model CNN (DeepGOPlus style) đã sẵn sàng chiến đấu!


## Train

In [3]:
# Chia tập train/val
from sklearn.model_selection import train_test_split
X_tr, X_val, Y_tr, Y_val = train_test_split(X_train_seq, Y_train, test_size=0.1, random_state=42)

# Dataset Loader
train_ds = data.TensorDataset(torch.from_numpy(X_tr).long(), torch.from_numpy(Y_tr).float())
val_ds = data.TensorDataset(torch.from_numpy(X_val).long(), torch.from_numpy(Y_val).float())
train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = data.DataLoader(val_ds, batch_size=BATCH_SIZE)

print("🚀 Bắt đầu Train CNN...")
for epoch in range(10): # Train nhanh 10 epoch
    model.train()
    total_loss = 0
    for bx, by in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
        bx, by = bx.to(device), by.to(device)
        optimizer.zero_grad()
        outputs = model(bx)
        loss = criterion(outputs, by)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    # Validate
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for bx, by in val_loader:
            bx, by = bx.to(device), by.to(device)
            val_loss += criterion(model(bx), by).item()
            
    print(f"Epoch {epoch+1}: Train Loss = {total_loss/len(train_loader):.4f} | Val Loss = {val_loss/len(val_loader):.4f}")

# Lưu model
torch.save(model.state_dict(), MODEL_PATH)
print("✅ Train xong & Đã lưu model!")

🚀 Bắt đầu Train CNN...


Epoch 1: Train Loss = 0.0173 | Val Loss = 0.0153


Epoch 2: Train Loss = 0.0156 | Val Loss = 0.0149


Epoch 3: Train Loss = 0.0153 | Val Loss = 0.0146


Epoch 4: Train Loss = 0.0150 | Val Loss = 0.0144


...

Epoch 6: Train Loss = 0.0144 | Val Loss = 0.0144


Epoch 7: Train Loss = 0.0141 | Val Loss = 0.0143


Epoch 8: Train Loss = 0.0138 | Val Loss = 0.0144


Epoch 9: Train Loss = 0.0135 | Val Loss = 0.0147


Epoch 10: Train Loss = 0.0132 | Val Loss = 0.0149
✅ Train xong & Đã lưu model!


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm

# --- CẤU HÌNH ---
TEST_FASTA = "/workspace/data/Test/testsuperset.fasta"
OUTPUT_CNN_RAW = "submission_cnn_raw.tsv"
BATCH_SIZE = 128

# 1. Encode tập Test (Dùng lại hàm get_one_hot_data cũ)
print("⏳ Đang mã hóa One-Hot cho tập Test (sẽ mất 1-2 phút)...")
# Hàm get_one_hot_data đã khai báo ở cell trước
X_test_seq, test_ids = get_one_hot_data(TEST_FASTA)

# Chuyển sang Tensor
test_dataset = TensorDataset(torch.from_numpy(X_test_seq).long())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"✅ Đã load {len(test_ids)} protein test.")

# 2. Dự đoán (Inference)
print("🚀 Đang chạy model CNN trên tập Test...")
model.eval()
output_lines = []
# Danh sách nhãn (top_terms) đã có từ cell trước

all_probs = []

with torch.no_grad():
    for bx in tqdm(test_loader):
        bx = bx[0].to(device)
        logits = model(bx)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_probs.append(probs)

# Gộp tất cả batch lại
all_probs = np.vstack(all_probs)

# 3. Ghi file (Lấy Top 50 để nhẹ file)
print(f"💾 Đang ghi kết quả xuống {OUTPUT_CNN_RAW}...")
for i, pid in enumerate(tqdm(test_ids)):
    # Lấy xác suất của protein i
    prob_row = all_probs[i]
    
    # Lọc Top 50 nhãn cao nhất
    top_indices = np.argsort(prob_row)[-50:]
    
    for idx in top_indices:
        score = prob_row[idx]
        # Chỉ lấy nếu score > 0.01
        if score > 0.01:
            term = top_terms[idx] # Lấy tên GO Term
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")

with open(OUTPUT_CNN_RAW, "w") as f:
    f.write("\n".join(output_lines))

print("✅ Đã có file dự đoán thô của CNN!")

⏳ Đang mã hóa One-Hot cho tập Test (sẽ mất 1-2 phút)...
⏳ Encoding /workspace/data/Test/testsuperset.fasta...


224309it [00:04, 48922.21it/s]


✅ Đã load 224309 protein test.
🚀 Đang chạy model CNN trên tập Test...


100%|██████████| 1753/1753 [00:18<00:00, 93.12it/s]


💾 Đang ghi kết quả xuống submission_cnn_raw.tsv...


100%|██████████| 224309/224309 [00:07<00:00, 28310.53it/s]


✅ Đã có file dự đoán thô của CNN!


In [5]:
# --- CẤU HÌNH ---
INPUT_FILE = "submission_cnn_raw.tsv"
OUTPUT_FILE = "submission_cnn_propagated.tsv"

# (Đảm bảo đã cài obonet: pip install obonet networkx)
import obonet
import networkx
import os

# Load OBO
OBO_PATH = "/workspace/data/go-basic.obo"
if not os.path.exists(OBO_PATH):
    OBO_PATH = "/workspace/data/Train/go-basic.obo"

print(f"📖 Reading OBO: {OBO_PATH}")
graph = obonet.read_obo(OBO_PATH)
ancestors_map = {}
for node in graph.nodes():
    try:
        ancestors_map[node] = networkx.descendants(graph, node)
    except:
        pass

# Đọc file & Propagate
print("🚀 Propagating scores...")
preds = {}
with open(INPUT_FILE) as f:
    for line in f:
        p, t, s = line.strip().split('\t')
        if p not in preds: preds[p] = {}
        preds[p][t] = float(s)

final_lines = []
for pid, scores in tqdm(preds.items()):
    new_scores = scores.copy()
    for term, score in scores.items():
        if term in ancestors_map:
            for parent in ancestors_map[term]:
                new_scores[parent] = max(new_scores.get(parent, 0.0), score)
    
    # Ghi lại (Top 70)
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)[:70]
    for term, score in sorted_terms:
        if score > 0.01:
            final_lines.append(f"{pid}\t{term}\t{score:.3f}")

with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(final_lines))
print(f"✅ Đã xong file CNN chuẩn: {OUTPUT_FILE}")

📖 Reading OBO: /workspace/data/Train/go-basic.obo
🚀 Propagating scores...


100%|██████████| 224305/224305 [00:12<00:00, 17964.95it/s]


✅ Đã xong file CNN chuẩn: submission_cnn_propagated.tsv


In [6]:
# --- CẤU HÌNH ---
# 1. File KNN xịn nhất của bạn (File đạt 0.256)
# Hãy thay tên file này bằng tên file tốt nhất bạn đang có!
FILE_KNN = "submission_level11_IA_official.tsv" 
# Nếu bạn lưu tên khác (vd submission_v11_B_bold.tsv) thì sửa lại nhé!

# 2. File CNN vừa tạo
FILE_CNN = "submission_cnn_propagated.tsv"

# 3. File nộp cuối cùng
OUTPUT_ENSEMBLE = "submission_FINAL_FUSION_0.35.tsv"

# Trọng số (Weights)
W_KNN = 0.60
W_CNN = 0.40

print("⏳ Đang load 2 file để hợp thể...")
preds_knn = {}
with open(FILE_KNN) as f:
    for line in tqdm(f, desc="Loading KNN"):
        p, t, s = line.strip().split('\t')
        preds_knn[(p, t)] = float(s)

preds_cnn = {}
with open(FILE_CNN) as f:
    for line in tqdm(f, desc="Loading CNN"):
        p, t, s = line.strip().split('\t')
        preds_cnn[(p, t)] = float(s)

# Trộn
print("🚀 FUSION: Mixing KNN (Sequence Sim) & CNN (Motif)...")
all_keys = set(preds_knn.keys()) | set(preds_cnn.keys())
output_lines = []

for key in tqdm(all_keys):
    pid, term = key
    s1 = preds_knn.get(key, 0.0)
    s2 = preds_cnn.get(key, 0.0)
    
    # Weighted Average
    final_score = (s1 * W_KNN) + (s2 * W_CNN)
    
    if final_score > 0.01:
        output_lines.append(f"{pid}\t{term}\t{final_score:.3f}")

with open(OUTPUT_ENSEMBLE, "w") as f:
    f.write("\n".join(output_lines))

print(f"🎉 XONG! File '{OUTPUT_ENSEMBLE}' là niềm hy vọng cuối cùng.")
print("👉 Hãy nộp file này. Sự kết hợp giữa Global (ESM2) và Local (CNN) thường đẩy điểm lên rất cao!")

⏳ Đang load 2 file để hợp thể...


Loading KNN: 14386016it [00:08, 1712368.83it/s]
Loading CNN: 14009920it [00:08, 1708294.71it/s]


🚀 FUSION: Mixing KNN (Sequence Sim) & CNN (Motif)...


100%|██████████| 23460696/23460696 [00:27<00:00, 859221.14it/s]


🎉 XONG! File 'submission_FINAL_FUSION_0.35.tsv' là niềm hy vọng cuối cùng.
👉 Hãy nộp file này. Sự kết hợp giữa Global (ESM2) và Local (CNN) thường đẩy điểm lên rất cao!
